# Molecular Toxicity Prediction (Tox21) using GCN

In this notebook, we implement and train a Graph Convolutional Network for predicting molecular toxicity based on the Tox21 dataset.

## 1. Environment Setup in Colab

Run the following code to install PyTorch Geometric and other dependencies:

In [ ]:
import os
import shutil

# This cell is intended to be run only in Google Colab
if 'COLAB_GPU' in os.environ:
    print("Running in Colab...")

    # 1. Remove existing folder if it already exists
    repo_name = 'gnn-molecule-prediction'
    repo_path = os.path.join('/content', repo_name)

    if os.path.exists(repo_path):
        shutil.rmtree(repo_path)
        print(f"Removed existing directory: {repo_path}")
    else:
        print(f"No existing directory found: {repo_path}")

    # 2. Clone the GitHub repository
    %cd /content
    !git clone https://github.com/sth-s/gnn-molecule-prediction.git

    # 3. Change working directory to the project root
    %cd gnn-molecule-prediction

    # 4. Install Conda support in Colab and restart runtime
    !pip install -q condacolab
    import condacolab
    condacolab.install()  # This will automatically restart the runtime
    !conda env update -n base -f environment.yml
    %cd gnn-molecule-prediction

else:
    os.chdir('../')
    print(f"Changed working directory to: {os.getcwd()}")


## 2. Import Libraries

In [ ]:
# PyTorch and PyG
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, Dataset
from torch_geometric.nn import GCNConv, global_mean_pool
from torch_geometric.loader import DataLoader

# Chemistry and data processing
from rdkit import Chem
import numpy as np

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Evaluation and splitting
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc, roc_curve
from sklearn.model_selection import train_test_split

# Our custom data loader
from src.data_utils import load_tox21
# Import our custom model and training functions
from src.model import GCN
from src.train import train_epoch, evaluate

# Set visualization style
sns.set_theme(style="whitegrid")
sns.set_palette('muted')

# Check CUDA availability
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")


## 3. Loading and Preparing Tox21 Data

### 3.1 About the Data Loading Module

We use a custom developed `load_tox21` module from the `src.data_utils` package, which provides the following features:

- **Automatic data downloading**: if the `tox21.csv` file is missing, the module will automatically download it from the official source
- **SMILES to graphs conversion**: each molecule is converted into a graph with node and edge attributes
- **Result caching**: results are saved to a cache to speed up subsequent runs
- **Flexible configuration**: you can specify the data path, file name, target columns, and other parameters

Detailed information about the `load_tox21` function and usage examples are available in the project's README.

In [ ]:
# Load the Tox21 dataset using our custom function
# If the tox21.csv file is missing, it will be automatically downloaded
dataset = load_tox21(
    root="data/Tox21",           # root directory for the data
    filename="tox21.csv",        # name of the CSV file
    smiles_col="smiles",         # column containing SMILES strings
    mol_id_col="mol_id",         # column containing molecule IDs
    cache_file="data.pt",        # name of the cache file
    recreate=False,              # Use existing processed file if available
    auto_download=True           # automatically download if missing
)

print(f"Type of dataset object: {type(dataset)}")
try:
    print(f"dataset._slices: {dataset._slices}")
except AttributeError:
    print(f"dataset has no attribute _slices")
print(f"Total graphs (len(dataset)): {len(dataset)}")

# Check if we can access the first item in the dataset
if len(dataset) > 0:
    print("Attempting to access dataset[0]...")
    first_item = None
    try:
        first_item = dataset[0]
        print(f"dataset[0] type: {type(first_item)}")
        if first_item is not None:
            print(f"Number of node features: {first_item.x.shape[1]}")
            if hasattr(first_item, 'mol_id'):
                 print(f"Mol ID for the first molecule: {first_item.mol_id}")
            else:
                 print(f"Mol ID attribute not found in first_item.")
            
            # Check the presence of y and y_mask
            if hasattr(first_item, 'y'):
                print(f"Task labels (y) shape: {first_item.y.shape}")
                print(f"Task labels (y): {first_item.y}")
            if hasattr(first_item, 'y_mask'):
                print(f"Task mask (y_mask) shape: {first_item.y_mask.shape}")
                print(f"Task mask (y_mask): {first_item.y_mask}")
        else:
            print(f"dataset[0] is None.")
    except Exception as e:
        print(f"Error accessing dataset[0] or its attributes: {e}")
else:
    print("Dataset is empty (len(dataset) is 0).")


### 3.3 Verify mol_id and Dataset Properties (Revised)

In [ ]:
# Verify that mol_id is present and check dataset properties
print("\nRunning second verification cell:")
if len(dataset) > 0:
    data_example = None
    try:
        data_example = dataset[0] # This might be None if the previous cell showed it was None
    except Exception as e:
        print(f"Error getting dataset[0] in second cell: {e}")
    
    if data_example is not None:
        print("Example graph from the dataset (after potential reprocessing):")
        print(data_example)
        
        if hasattr(data_example, 'mol_id'):
            print(f"\nMol ID for the first molecule: {data_example.mol_id}")
        else:
            print("\nmol_id attribute not found in the first molecule (data_example).")
        
        print(f"Edge index dimensions: {data_example.edge_index.shape}")
        print(f"Number of atoms (nodes): {data_example.num_nodes}")
        print(f"Task labels (y): {data_example.y}")
        print(f"Task mask (y_mask): {data_example.y_mask}")
    elif len(dataset) > 0: # dataset[0] was None but dataset is not empty
        print("data_example (dataset[0]) is None, cannot print details.")
else:
    print("Dataset is empty in second verification cell.")

In [ ]:
# Analysis of graph properties in the dataset
if len(dataset) > 0:
    data_example = dataset[0]
    print("Example graph from the dataset:")
    print(data_example)
    print(f"Edge index dimensions: {data_example.edge_index.shape}")
    print(f"Number of atoms (nodes): {data_example.num_nodes}")
    print(f"Task labels (y): {data_example.y}")

    # Dataset statistics
    nodes_count = []
    edges_count = []
    for i in range(min(1000, len(dataset))):
        data = dataset[i]
        nodes_count.append(data.num_nodes)
        edges_count.append(data.edge_index.shape[1])
    
    print(f"\nGraph statistics (based on a sample of {len(nodes_count)} molecules):")
    print(f"Average number of atoms: {np.mean(nodes_count):.2f} ± {np.std(nodes_count):.2f}")
    print(f"Average number of bonds: {np.mean(edges_count)/2:.2f} ± {np.std(edges_count)/2:.2f}")
    print(f"Min/max atoms: {np.min(nodes_count)}/{np.max(nodes_count)}")
    print(f"Min/max bonds: {np.min(edges_count)/2:.0f}/{np.max(edges_count)/2:.0f}")

In [ ]:
# Visualization of atom and bond count distributions
if 'nodes_count' in locals():
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))
    
    ax1.hist(nodes_count, bins=30, alpha=0.7, color='skyblue')
    ax1.set_title('Distribution of Atom Counts')
    ax1.set_xlabel('Number of atoms')
    ax1.set_ylabel('Number of molecules')
    
    ax2.hist([e/2 for e in edges_count], bins=30, alpha=0.7, color='salmon')
    ax2.set_title('Distribution of Bond Counts')
    ax2.set_xlabel('Number of bonds')
    ax2.set_ylabel('Number of molecules')
    
    plt.tight_layout()
    plt.show()

In [ ]:
# Split into train/test (80/20)
torch.manual_seed(42)  # for reproducibility
train_len = int(0.8 * len(dataset))
train_dataset, test_dataset = torch.utils.data.random_split(
    dataset, [train_len, len(dataset) - train_len]
)

# Create DataLoader
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader  = DataLoader(test_dataset, batch_size=32, shuffle=False)

print(f"Total graphs: {len(dataset)}")
print(f"Training set: {len(train_dataset)} graphs")
print(f"Test set: {len(test_dataset)} graphs")
print(f"Batch size: {train_loader.batch_size}, number of batches in training set: {len(train_loader)}")

## 4. Define GCN Model

In [ ]:
# Create a GCN model instance using our implementation from src/model.py
model = GCN(
    in_channels=dataset.num_node_features,
    hidden_channels=64,
    num_classes=dataset.num_classes,
    num_layers=2,
    dropout=0.5
).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

print(f"Model created with {dataset.num_node_features} input features and {dataset.num_classes} output classes")
print(model)

## 5. Training and Evaluation

In [ ]:
# Training and evaluation using our functions from src/train.py
num_epochs = 20
best_val_roc = 0
best_model_state = None
history = {'train_loss': [], 'val_roc': [], 'val_pr': []}

print("Starting training...")
for epoch in range(1, num_epochs + 1):
    # Train for one epoch
    loss = train_epoch(model, train_loader, optimizer, device)
    
    # Evaluate the model
    val_metrics = evaluate(model, test_loader, device)
    val_roc = val_metrics['roc_auc']
    val_pr = val_metrics['pr_auc']
    
    # Save metrics history
    history['train_loss'].append(loss)
    history['val_roc'].append(val_roc)
    history['val_pr'].append(val_pr)
    
    # Print progress
    print(f"Epoch {epoch:02d}: Loss = {loss:.4f}, ROC AUC = {val_roc:.4f}, PR AUC = {val_pr:.4f}")
    
    # Save best model state
    if val_roc > best_val_roc:
        best_val_roc = val_roc
        best_model_state = model.state_dict().copy()
        print(f"  → New best model saved! ROC AUC: {best_val_roc:.4f}")

# Load the best model state for final evaluation
if best_model_state is not None:
    model.load_state_dict(best_model_state)
    print(f"Loaded best model with validation ROC AUC: {best_val_roc:.4f}")

# Perform final evaluation
final_metrics = evaluate(model, test_loader, device)
print("\nFinal Evaluation Results:")
print(f"Test ROC AUC: {final_metrics['roc_auc']:.4f}")
print(f"Test PR AUC: {final_metrics['pr_auc']:.4f}")

# Save the best model
torch.save(best_model_state, 'best_gcn_model.pt')

## 6. Results Visualization

In [ ]:
# Visualize learning curves
plt.figure(figsize=(12, 4))

# Plot training loss
plt.subplot(1, 2, 1)
plt.plot(range(1, len(history['train_loss'])+1), history['train_loss'], 'o-', label='Training Loss')
plt.title('Training Loss per Epoch')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.grid(True)
plt.legend()

# Plot validation ROC AUC
plt.subplot(1, 2, 2)
plt.plot(range(1, len(history['val_roc'])+1), history['val_roc'], 'o-', label='ROC AUC')
plt.plot(range(1, len(history['val_pr'])+1), history['val_pr'], 'o-', label='PR AUC')
plt.title('Validation Metrics per Epoch')
plt.xlabel('Epoch')
plt.ylabel('AUC')
plt.grid(True)
plt.legend()

plt.tight_layout()
plt.show()

# Plot ROC and PR curves for the three best endpoints
model.eval()

# Collect predictions for each task
all_true = {}
all_pred = {}

with torch.no_grad():
    for batch in test_loader:
        batch = batch.to(device)
        out = model(batch).sigmoid().detach().cpu()
        mask = batch.y_mask.cpu()
        y = batch.y.cpu()
        
        for task in range(out.size(1)):
            idx = mask[:, task]
            if idx.sum() == 0:
                continue
                
            true = y[idx, task].numpy()
            pred = out[idx, task].numpy()
            
            all_true.setdefault(task, []).append(true)
            all_pred.setdefault(task, []).append(pred)

# Calculate AUC for each task
task_metrics = {}
for task in all_true:
    t = np.concatenate(all_true[task])
    p = np.concatenate(all_pred[task])
    
    # Skip if only one class is present
    if len(np.unique(t)) < 2:
        continue
        
    # Calculate ROC AUC
    roc_auc = roc_auc_score(t, p)
    
    # Calculate PR AUC
    prec, rec, _ = precision_recall_curve(t, p)
    pr_auc = auc(rec, prec)
    
    # Store task metrics and raw predictions for plotting
    task_metrics[task] = {
        'roc_auc': roc_auc, 
        'pr_auc': pr_auc,
        'true': t,
        'pred': p
    }

# Find the three best endpoints based on ROC AUC
best_tasks = sorted(task_metrics.keys(), key=lambda x: task_metrics[x]['roc_auc'], reverse=True)[:3]
print(f"Three best endpoints (task indices): {best_tasks}")

# Plot ROC curves for the three best endpoints
plt.figure(figsize=(12, 5))

# ROC Curves
plt.subplot(1, 2, 1)
for task in best_tasks:
    t = task_metrics[task]['true']
    p = task_metrics[task]['pred']
    fpr, tpr, _ = roc_curve(t, p)
    plt.plot(fpr, tpr, label=f'Task {task} (AUC = {task_metrics[task]["roc_auc"]:.3f})')

plt.plot([0, 1], [0, 1], 'k--')
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curves for Best Endpoints')
plt.legend()
plt.grid(True)

# PR Curves
plt.subplot(1, 2, 2)
for task in best_tasks:
    t = task_metrics[task]['true']
    p = task_metrics[task]['pred']
    prec, rec, _ = precision_recall_curve(t, p)
    plt.plot(rec, prec, label=f'Task {task} (AUC = {task_metrics[task]["pr_auc"]:.3f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('PR Curves for Best Endpoints')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

## 7. Conclusion and Findings

# TODO: Add analysis of results and conclusions